In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets tqdm pandas

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install sentencepiece transformers wandb

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from itertools import chain
from string import punctuation
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)
pd.set_option('display.max_colwidth', None)
df = pd.read_parquet("/content/drive/MyDrive/train-00000-of-00001.parquet")
df.shape

(216318, 6)

In [ ]:
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
  )
from torch.utils.data import Dataset, DataLoader

In [ ]:
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [ ]:
def calc_token_len(example):
  return len(tokenizer(example).input_ids)

In [ ]:
train_df = df
test_df = pd.read_parquet("/content/drive/MyDrive/test-00000-of-00001.parquet")

In [ ]:
test_df.head()

,lang,sentence,modified,transformation,sec_transformation,__index_level_0__
0,es,Ella tiene la opción de quedarse o irse.,fix grammar: Ella tiene lo opción de quedarse o irse.,NounGenderAgreementChangerDestroyer,None,54704
1,es,Se puso colorada cuando le dijeron un piropo.,fix grammar: Se puso colorada cuando le dijeron una piropo.,GenderDeterminerDestroyer,None,21839
2,fr,La confrontation des témoins qui l'avaient rencontrée le jour de sa mort devait aider l'inspecteur.,fix grammar: L'confrontation des témoins qui le avaient rencontrée l'jour de sa mort devait aider le inspecteur.,ApostropheChangerDestroyer,None,11751
3,de,"Um bessere Aufklärungsergebnisse zu bekommen, befahl General von Gronau einen begrenzten Angriff seines Korps nach Westen.","fix grammar: Um bessere Aufklärungsergebnisse zu bekommen, befahl General von Gronau der begrenzten Angriff seines Korpse nach Westen.",DetNounNumberDestroyer,ArticleDefChangerDestroyer,8787
4,de,"Ich fürchte, dass er mich nicht mehr liebt.","fix grammar: Ich furchte, dass er mich nicht mehr liebt.",DeRemoveSpecialCharactersDestroyer,None,9910


In [ ]:
test_df['input_token_len'] = test_df['modified'].apply(calc_token_len)

In [ ]:
!pip install datasets
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64

    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['modified'], example['sentence']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length,
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [ ]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[121])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 12
attention_mask 12
labels 10
{'input_ids': [2210, 19519, 10, 3885, 6988, 20, 22089, 259, 4514, 9, 109, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [1138, 6988, 20, 22089, 259, 4514, 9, 109, 5, 1]}


In [ ]:
!pip install rouge_score

In [ ]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

<ipython-input-15-76fa4d2ef6b0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
!pip install accelerate -U

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [ ]:
!pip install transformers[torch]

In [ ]:
pip install transformers[torch] -U

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="/content/drive/MyDrive/multilingual-gec/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = False,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model,
                args=args,
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.save_model('/content/drive/MyDrive/multilingual_gec')

In [ ]:
trainer.train()
trainer.save_model('/content/drive/MyDrive/multilingual_gec')

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_device

'cuda'

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = '/content/drive/MyDrive/multilingual-gec/weights/checkpoint-2000'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def get_correct_text_recursively(model, tokenizer, input_text):
    texts = correct_grammar(input_text, 4)
    possible_text = {}
    for text in texts:
        temp = correct_grammar(text, 4)
        temp.append(text)
        for t in temp:
            if t in possible_text.keys():
                possible_text[t] +=1
            else:
                possible_text[t] = 1

    return max(possible_text, key=possible_text.get)

In [ ]:
get_correct_text_recursively(model, tokenizer, "fix grammar: I have good reason for avoiding Tom.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'I have good reason to avoid Tom.'

In [ ]:
import pandas as pd
test_cases = pd.read_parquet("/content/drive/MyDrive/test-00000-of-00001.parquet")
test_cases

,lang,sentence,modified,transformation,sec_transformation,__index_level_0__
0,es,Ella tiene la opción de quedarse o irse.,fix grammar: Ella tiene lo opción de quedarse ...,NounGenderAgreementChangerDestroyer,None,54704
1,es,Se puso colorada cuando le dijeron un piropo.,fix grammar: Se puso colorada cuando le dijero...,GenderDeterminerDestroyer,None,21839
2,fr,La confrontation des témoins qui l'avaient ren...,fix grammar: L'confrontation des témoins qui l...,ApostropheChangerDestroyer,None,11751
3,de,"Um bessere Aufklärungsergebnisse zu bekommen, ...",fix grammar: Um bessere Aufklärungsergebnisse ...,DetNounNumberDestroyer,ArticleDefChangerDestroyer,8787
4,de,"Ich fürchte, dass er mich nicht mehr liebt.","fix grammar: Ich furchte, dass er mich nicht m...",DeRemoveSpecialCharactersDestroyer,None,9910
...,...,...,...,...,...,...
2181,es,Se ha incrementado el servicio de transporte p...,fix grammar: Se ha incrementado el servicio de...,RemoveAccentsDestroyer,None,14261
2182,fr,Où avez-vous trouvé ce couteau ?,fix grammar: Où as-vous trouvait ce couteau ?,PersonVerbDisagreement,FrVerbNumberChanger,2781
2183,de,Am frühen Morgen des 27. Januar 1945 trafen de...,fix grammar: Am frühen Morgen des 27. Januar 1...,DetNounNumberDestroyer,None,4643
2184,es,Medio ambiente 2010: el futuro en nuestras manos.,fix grammar: Medio anbiente 2010: el futuro en...,SimilarSoundsDestroyer,None,29420


In [ ]:
get_correct_text_recursively(model, tokenizer, "Ellos estudio mucho para el examen")

{'Ellos estudio mucho para el examen.': 3,
 'Ellos se estudio mucho para el examen.': 1,
 'Ellos estudio mucha para el examen.': 1,
 'Ellos sestudio mucho para el examen.': 1,
 'Ellos estudio mucho para el examene.': 2,
 'Ellos se estudio mucho para el examene.': 1,
 'Ellos sestudio mucho para el examene.': 1,
 'Ellos estudio mucho para el examen del ltimo.': 3,
 'Ellos estudio mucho para el examen del lder.': 1,
 'Ellos estudio mucho para el examen del lneo.': 1,
 'Ellos estudio mucho para el examene del ltimo.': 3,
 'Ellos estudio mucho para el examene del lder.': 1,
 'Ellos estudio mucho para el examene del lneo.': 1}

In [ ]:
for test in test_cases.iloc:
  res = get_correct_text_recursively(model, tokenizer, test["modified"])
  print("Input:", test["modified"])
  print("Actual:", test["sentence"])
  print("Predict:", res)
  print("------------------------------------------------------------------------------------")

Input: fix grammar: Ella tiene lo opción de quedarse o irse.
Actual: Ella tiene la opción de quedarse o irse.
Predict: Ella tiene la oportunidad de quedarse o irse.
------------------------------------------------------------------------------------
Input: fix grammar: Se puso colorada cuando le dijeron una piropo.
Actual: Se puso colorada cuando le dijeron un piropo.
Predict: Se puso colorado cuando le dijeron un piropo.
------------------------------------------------------------------------------------
Input: fix grammar: L'confrontation des témoins qui le avaient rencontrée l'jour de sa mort devait aider le inspecteur.
Actual: La confrontation des témoins qui l'avaient rencontrée le jour de sa mort devait aider l'inspecteur.
Predict: La confrontation des témoins qui le avaient rencontré le jour de sa mort devait aider le inspecteur.
------------------------------------------------------------------------------------
Input: fix grammar: Um bessere Aufklärungsergebnisse zu bekommen, 